# Uncertainty analysis of measured irradiance

This script defines the functions used to perform a measurement uncertainty analysis on the GHI DHI, DNI, and albedo measurements retrieved from the pyranometers and pyrheliometer at the test site. These functions have been developed in a master's thesis written in the spring of 2021 and can be found on the GitHub repository  [Performance-modeling-of-Bifacial-PV-Power-Plants-in-a-Nordic-Climate](https://github.com/Dina-CM/Performance-modeling-of-Bifacial-PV-Power-Plants-in-a-Nordic-Climate).

After the analysis was performed on data, the GHI DHI, DNI, and albedo measurements were corrected with the measurement uncertainty. The results were implemented into dataframes that were used to create weather files.

### Importing the necessary packages

In [ ]:
import numpy as np
import pandas as pd
import pvlib

### The functions for measurement uncertianty

uncertainty_measured_SMP10() were used for the pyranometer measurements, in other words the GHI, DHI and albedo.
uncertainty_measured_SHP1() were used for the pyrheliometer measurements, in other words the DNI.

In [ ]:
def uncertainty_measured_SMP10(data, col_name, latitude, longitude, tz, altitude):
    
    S = 10.91                               # [$\micro$V/(W/m²)] Sensitivity
    zero_A = (7 / np.sqrt(3)) / 2           # [W/m²] Offset - Thermal radiation: rectangular distribution, one-sided
    zero_B = 2 / np.sqrt(3)                 # [W/m²] Offset - Temperature dependence: rectangular distribution, symmetric
    dir_resp = 10 / np.sqrt(3)              # [W/m²] Directional response: rectangular distribution, symmetric
    calibration = 1.39 / 3                  # [%] Calibration uncertainty: normal distribution, symmetric
    non_stability = 0.5 / np.sqrt(3)        # [%] Stability deviation per year: rectangular, symmetric
    non_linearity = 0.2 / np.sqrt(3)        # [%] Non-linearity: rectangular, symmetric
    tempe_resp = 1 / np.sqrt(3)             # [%] Temperature response: rectangular, symmetric
    datalogger = 0
    
    site = pvlib.location.Location(latitude, longitude, tz, altitude, 'Kjeller')
    ephem = site.get_solarposition(data.index)

    dir_resp_adjusted = (data[col_name]*dir_resp)/(1000*np.radians(ephem['apparent_elevation']))
    zero_A_adjusted = zero_A/100 * data[col_name]
    zero_B_adjusted = zero_B /100 * data[col_name]
    datalogger_adjusted = datalogger/100 * data[col_name]

    cs = np.abs(-(data[col_name]*S)/S**2)
    u_s = np.sqrt((calibration/100 * S)**2 + (non_stability/100 * S)**2 + (non_linearity/100 * S)**2 + (tempe_resp/100 * S)**2)
    cv = np.abs(1/S)
    u_v = np.sqrt(datalogger_adjusted**2)
    Uc1 = (cv*u_v)**2 + (cs*u_s)**2
    Uc2 = np.sqrt(zero_A_adjusted**2 + zero_B_adjusted**2 + (dir_resp_adjusted )**2 )
    Uc = np.sqrt(Uc1 + Uc2)
    UE = 2*Uc # k = 2 coverage factor
    data['UE_' + col_name] = UE
    
    return data

In [ ]:
def uncertainty_measured_SHP1(data, col_name):
        S = 9.08                                # [$\micro$V/(W/m²)] Sensitivity
        Offset = (1 / np.sqrt(3)) / 2           # [W/m²] Offset: rectangular distribution, symmetric
        tempe_resp = 0.5/np.sqrt(3)             # [%] Temperature response: rectangular, symmetric
        non_linearity = 0.2 / np.sqrt(3)        # [%] Non-linearity: rectangular, symmetric
        non_stability = 0.5 / np.sqrt(3)        # [%] Stability deviation per year: rectangular, symmetric
        calibration = 0                         # [%] Calibration uncertainty: normal distribution, symmetric
        datalogger = 0

        offsett_adjusted = Offset / 100 * data[col_name]
        datalogger_adjusted = datalogger / 100 * data[col_name]

        cs = np.abs(-(data[col_name] * S) / S ** 2)
        u_s = np.sqrt(((non_linearity / 100 * S) ** 2 + (non_stability/100 * S)**2 + (tempe_resp / 100 * S) ** 2) 
                      + (calibration/100 * S)**2 )
        
        cv = np.abs(1 / S)
        u_v = np.sqrt(datalogger_adjusted ** 2)
        Uc1 = (cv * u_v) ** 2 + (cs * u_s) ** 2
        Uc2 = np.sqrt(offsett_adjusted ** 2 )
        Uc = np.sqrt(Uc1 + Uc2)
        UE = 2 * Uc + 5 + (0.025 * data[col_name])  # k = 2 
        data['UE_' + col_name] = UE
        
        return data

### Importing data for the uncertainty analysis

In [ ]:
data = pd.read_csv(r'path.../filename.csv', sep=';')                     # Importing the file with data
data = data.set_index('Timestamp')                                       # Setting the timestamps to be the index
data.index = pd.to_datetime(data.index)                                  # Converting index to date-time
data = data.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')   # Localize timezone and handling daylight saving time
data.loc[data.index.dropna()]                                            # Removing NaN values
data = data.tz_convert('UTC')                                            # Converting the timezone to UTC

# Renaming columns in the dataframe
data.rename(columns = {'Roof_albedo_downwell [W/m²]':'albedo_ned',
                       'Roof_albedo_upwell [W/m²]':'albedo_opp',
                       'Solys_ghi [W/m²]':'GHI',
                       'Solys_dhi [W/m²]':'DHI',
                       'Solys_dni [W/m²]':'DNI'}, inplace=True)

### Creating a dataframe and inserting the necessary data

In [ ]:
irradiance = pd.DataFrame()

irradiance['GHI'] = data['GHI']
irradiance['DHI'] = data['DHI']
irradiance['DNI'] = data['DNI']
irradiance['Albedo_ned'] = data['albedo_ned']
irradiance['Albedo_opp'] = data['albedo_opp']
irradiance['Albedo'] = data['albedo_opp'] / data['albedo_ned']

### Performing uncertianty analysis on the data for the test site 

In [ ]:
lat = 59.97315     # Latitude of the position of the PV system  
lon = 11.05166     # Longitude of the position of the PV system 
tz = 0             # Wanted timezone, UTC 
alt = 214          # Altitude

data_uncertainty = uncertainty_measured_SMP10(irradiance, 'GHI', lat, lon, tz, alt) 
data_uncertainty = uncertainty_measured_SMP10(irradiance, 'DHI', lat, lon, tz, alt) 
data_uncertainty = uncertainty_measured_SMP10(irradiance, 'Albedo_opp', lat, lon, tz, alt) 
data_uncertainty = uncertainty_measured_SMP10(irradiance, 'Albedo_ned', lat, lon, tz, alt) 
data_uncertainty = uncertainty_measured_SHP1(innstråling, 'DNI') 

### Calculating the measurement uncertianty in albedo from the measurement uncertianty in upward and downward irradiance

In [ ]:
data_uncertainty['UE_albedo'] = (1/data_uncertainty['Albedo_ned']) * np.sqrt(((data_uncertainty['Albedo_opp'] /
                                                                               data_uncertainty['Albedo_ned']) * 
                                                                               data_uncertainty['UE_Albedo_opp'])**2 + 
                                                                              (data_uncertainty['UE_Albedo_ned'])**2)

### Calculating the lower and upper limit of irradiance where the measurement uncertianty is taken into account by subdtracting and adding to the measured irradiance

In [ ]:
data_uncertainty['min GHI'] = data_uncertainty['GHI'] - data_uncertainty['UE_GHI']
data_uncertainty['min DNI'] = data_uncertainty['DNI'] - data_uncertainty['UE_DNI']
data_uncertainty['min DHI'] = data_uncertainty['DHI'] - data_uncertainty['UE_DHI']
data_uncertainty['Albedo_min'] = data_uncertainty['Albedo'] - data_uncertainty['UE_albedo']

data_uncertainty['maks GHI'] = data_uncertainty['GHI'] + data_uncertainty['UE_GHI']
data_uncertainty['maks DNI'] = data_uncertainty['DNI'] + data_uncertainty['UE_DNI']
data_uncertainty['maks DHI'] = data_uncertainty['DHI'] + data_uncertainty['UE_DHI']
data_uncertainty['Albedo_max'] = data_uncertainty['Albedo'] + data_uncertainty['UE_albedo']

### Saving the upper and lower limit of irradiance in two dataframes 

In [ ]:
minimal = pd.DataFrame()
maximal = pd.DataFrame()

minimal['GHI'] = data_uncertainty['min GHI']
minimal['DHI'] = data_uncertainty['min DHI']
minimal['DNI'] = data_uncertainty['min DNI']
minimal['Albedo'] = data_uncertainty['Albedo_min']

minimal.to_csv(r'path...\filename.csv')    

maximal['GHI'] = data_uncertainty['maks GHI']
maximal['DHI'] = data_uncertainty['maks DHI']
maximal['DNI'] = data_uncertainty['maks DNI']
maximal['Albedo'] = data_uncertainty['Albedo_max']

maximal.to_csv(r'path...\filename.csv')    

The two dataframes minimal and maximal can be imported as data to make a weather file.